In [109]:
import pickle
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV

from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import RMSprop


from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

from sklearn.metrics import accuracy_score
from keras.layers import Concatenate

import keras
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [84]:
class PlotLearning(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="accuracy")
        ax2.plot(self.x, self.val_acc, label="validation accuracy")
        ax2.legend()
        
        plt.show();
        
plot = PlotLearning()

In [85]:
y = np.load("y.npy")
X = np.load("X.npy")

In [86]:
X_notes = X.T[:28].T
y_notes = y
X_chords = X.T[56:].T

In [87]:
X_chords.shape

(10204, 28)

In [88]:
y_notes.shape

(10204,)

In [89]:
X_chords = np.column_stack((X_chords, y_notes.T))
X_chords = X_chords.reshape((-1,1))

In [90]:
chords = np.unique(X_chords)
len(chords)

36

In [91]:
coded_X_chords = []

for x_chord in X_chords:
    for i, chord in enumerate(chords):
        if x_chord == chord:
            coded_X_chords.append(i)

            
            

In [92]:
data = [coded_X_chords[i:i + 7] for i in range(0, len(coded_X_chords)-6, 1)]
data = np.array(data)
data = np.reshape(data, (-1, 7))
print(data.shape)

(295910, 7)


In [93]:
data = np.array(data)

X_chords = data[:,0:6]
y_chords = data[:,6]

In [94]:
train_X, test_X, train_y, test_y = train_test_split(X_chords, y_chords, train_size=0.9, random_state=0)
print(test_X.shape)
print(train_X.shape)

(29591, 6)
(266319, 6)


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [128]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_notes, y_notes)

In [129]:
X_notes.shape

(10204, 28)

In [161]:
model_1 = Sequential()
model_1.add(Embedding(37, 20, input_length=6))
model_1.add(LSTM(20))
#model.add(Dense(37, activation='softmax'))
#model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#model.fit(train_X, train_y, epochs=1, batch_size=1, verbose=1, validation_data=(test_X, test_y))#, callbacks=[plot]);

In [162]:
model_2 = Sequential()
model_2.add(Embedding(15, 20, input_length=28))
model_2.add(LSTM(20))
#model.add(Dense(17, activation='softmax'))
#model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#model.fit(X_train_2, y_train_2, epochs=60, verbose=1, validation_data=(X_test_2, y_test_2), callbacks=[plot]);

In [163]:
merged_model = Sequential()
#merged_model.add(Dense( ? , input_shape=( ? , ? ), activation='sigmoid'))
merged_model.add(Concatenate([model_1, model_2]))
#merged_model.add(Dense( ? , activation='softmax'))
#merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#merged_model.fit([X_1, X_2], y, batch_size=200, epochs=10, verbose=1)

ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.